In [ ]:
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [ ]:
raster=r"E:/rastermunicpial/atlacomulco_capa.tif"
src=rasterio.open(raster)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=10000)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy

In [ ]:
def ampliar_shape(shape,factor_ampliacion=2):
    """Amplifica el polygon de cada manzana con el fin de extrar imagenes sin perder informacion de la manzana"""
    # shape["geometry"]=shape["geometry"]
    shape['centroid']=shape.centroid
    geometry=[]
    clase=[]
    for i,polygon in enumerate(shape['geometry']):
        try:
            point=mapping(shape['centroid'][i]).get('coordinates')
        except:
            continue
        x=point[0]
        y=point[1]
        go=[]
        coodinates=mapping(polygon).get('coordinates')[0]
        for a in coodinates:
            x1=a[0]
            y1=a[1]
            x2=x+(x1-x)*factor_ampliacion
            y2=y+(y1-y)*factor_ampliacion
            go.append((x2,y2))
        geometry.append(Polygon(go))
#         clase.append(shape.loc[i,"clase_dete"])
    return gpd.GeoDataFrame(geometry=geometry,crs="3857")

In [ ]:
deteccion=r"C:\Users\ASUS\Desktop\DELEGACIONES_EDOMEX\Zona_Atlacomulco.shp"
deteccion_shp=gpd.read_file(deteccion).to_crs("3857")
deteccion_shp=deteccion_shp.dropna().reset_index(drop=True)
angulo_manzana=[]
for manzana in range(len(deteccion_shp)):
    proyecciones1=mapping(deteccion_shp['geometry'][manzana]).get('coordinates')
    angulos=[]
    d=[]
    poly=pd.DataFrame(proyecciones1[0])
    for point in range(1,len(poly)):
        d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
        angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
#     angulo_manzana.append(angulos[d.index(max(d))])
# deteccion_shp['angulo']=angulo_manzana
# deteccion_shp["geometry"]=deteccion_shp["geometry"].envelope
deteccion_shp1=ampliar_shape(deteccion_shp,factor_ampliacion=1)

In [ ]:
# shape=deteccion_shp.copy()
# shape['centroid']=shape.centroid
# factor_ampliacion=2
# geometry=[]
# clase=[]
# for i,polygon in enumerate(shape['geometry'].envelope):
#     try:
#         point=mapping(shape['centroid'][i]).get('coordinates')
#     except:
#         continue
#     x=point[0]
#     y=point[1]
#     go=[]
#     coodinates=mapping(polygon).get('coordinates')[0]
#     print(i)
#     for a in coodinates:
#         x1=a[0]
#         y1=a[1]
#         x2=x+(x1-x)*factor_ampliacion
#         y2=y+(y1-y)*factor_ampliacion
#         go.append((x2,y2))
#     geometry.append(Polygon(go))
# #         clase.append(shape.loc[i,"clase_dete"])
# gpd.GeoDataFrame(geometry=geometry,crs="3857")

In [ ]:
# shape=deteccion_shp.copy()
# shape['centroid']=shape.centroid
# point=mapping(shape['centroid'][0]).get('coordinates')
# point
# coodinates=mapping(polygon).get('coordinates')[0]
# for a in coodinates:
#     print(a)

In [ ]:
# shape.envelope

In [ ]:
def color_segmetation(img1):
    mask = cv2.GaussianBlur(img1, (7, 7), 3)
    f=np.where([mask[:,:,2]<160, img1[:,:,1]>16, img1[:,:,1]<100])
    va=[i for i,x in enumerate(np.array(f)[0,:]) if x==0]
    img1[list(f)[1][va],list(f)[2][va]]=0
    w,h,_=img1.shape
    mask=cv2.threshold(img1[:,:,0],180,255,cv2.THRESH_BINARY)[1]    
    mask = cv2.dilate(mask,(5,5))
    mask=cv2.bitwise_not(mask)
    contours,_ = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1 )
    geometry=[]
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        dif=np.max([x[0] for x in contours[0]],axis=0)-np.min([x[0] for x in contours[0]],axis=0)
        area2=dif[0]*dif[1]
        if area>=.6*area2 and area>dim and area<H*W:
            geometry.append(Polygon([[map_d(x[0][0],0,W,minx,maxx),map_d(x[0][1],H,0,miny,maxy)] for x in contours[i]]))

    shp_contorno=gpd.GeoDataFrame(geometry=geometry,crs=3857)
    shp_contorno["area"]=shp_contorno.area
    shp_contorno=shp_contorno.sort_values(by="area",ascending=False).reset_index(drop=True)[1:]
    # shp_contorno["geometry"]=shp_contorno["geometry"]
    # shp_contorno["a1"]=shp_contorno["geometry"].area
    # shp_contorno["a2"]=shp_contorno["geometry"].envelope.area
    # shp_contorno=shp_contorno[shp_contorno["a1"]>=.2*shp_contorno["a2"]]
#     shp_contorno=shp_contorno[shp_contorno["area"]>1000]
#     shp_contorno.plot()
    return shp_contorno

In [ ]:
redBajo1 = np.array([0, 0, int(.7*255)], np.uint8)
redAlto1 = np.array([179, 100, 255], np.uint8)
def color_segmetation(frame):
    frameHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    maskRed = cv2.inRange(frameHSV, redBajo1, redAlto1)
    maskRedvis = cv2.bitwise_and(frame, frame, mask= maskRed)
    _,thresh=cv2.threshold(maskRedvis[:,:,2],200,255,3)
    contours, hierarchy = cv2.findContours(thresh, 0, 2)
    geometry=[]
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        dif=np.max([x[0] for x in contours[0]],axis=0)-np.min([x[0] for x in contours[0]],axis=0)
        area2=dif[0]*dif[1]
        if area>10:# and area<H*W:
            geometry.append(Polygon([[map_d(x[0][0],0,W,minx,maxx),map_d(x[0][1],H,0,miny,maxy)] for x in contours[i]]))

    shp_contorno=gpd.GeoDataFrame(geometry=geometry,crs=3857)
    shp_contorno["area"]=shp_contorno.area
    shp_contorno=shp_contorno.sort_values(by="area",ascending=False).reset_index(drop=True)[1:]
    return shp_contorno

In [ ]:
# deteccion_shp2=deteccion_shp[(deteccion_shp["cve_cat"]=="0980422100000000")].reset_index(drop=True)
# deteccion_shp1=ampliar_shape(deteccion_shp2,factor_ampliacion=2)
# deteccion_shp1

In [11]:
import tqdm
import cv2
shp_contorno_res=gpd.GeoDataFrame()
s1=gpd.GeoDataFrame()
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    try:
        array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    except:
        continue
    four_images=[array[0],array[1],array[0]]
    stacked_images = np.stack(four_images, axis=-1)
    im=np.array(stacked_images)
    # cv2.namedWindow('maskRedvis',cv2.WINDOW_NORMAL)
    # cv2_imshow('maskRedvis', im)
    balck=np.full((im.shape[0],im.shape[1], 3), (0,0,0), dtype=np.uint8)
    im=np.moveaxis(im,-1,0)
    im=np.stack([np.abs(im[0]),im[1],im[2]])
    im=np.moveaxis(im,0,-1)
    im=cv2.GaussianBlur(im,(5,5),0)
    frame=im.copy()
    frameHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    maskRed = cv2.inRange(frameHSV, redBajo1, redAlto1)
    maskRedvis = cv2.bitwise_and(frame, frame, mask= maskRed)
    _,thresh=cv2.threshold(maskRedvis[:,:,2],200,255,3)
    contours, hierarchy = cv2.findContours(thresh, 0, 2)
    proyecciones=np.array(shapes[0].get("coordinates")[0])
    minx,miny=np.min(proyecciones,axis=0)
    maxx,maxy=np.max(proyecciones,axis=0)
    W=int(int((maxx-minx))*dim/10000)
    H=int(int((maxy-miny))*dim/10000)
    geometry=[]
    s=[]
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        dif=np.max([x[0] for x in contours[i]],axis=0)-np.min([x[0] for x in contours[i]],axis=0)
        area2=dif[0]*dif[1]
        if area>100:# and area<H*W:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            s.append(Polygon([[map_d(x[0],0,W,minx,maxx),map_d(x[1],H,0,miny,maxy)] for x in box]))
            geometry.append(Polygon([[map_d(x[0][0],0,W,minx,maxx),map_d(x[0][1],H,0,miny,maxy)] for x in contours[i]]))
    s2=gpd.GeoDataFrame(geometry=s,crs=3857)
    shp_contorno=gpd.GeoDataFrame(geometry=geometry,crs=3857)
    s2["area"]=shp_contorno.area
    shp_contorno["area"]=shp_contorno.area
    shp_contorno=shp_contorno.sort_values(by="area",ascending=False).reset_index(drop=True)
    shp_contorno_res=pd.concat([shp_contorno_res,shp_contorno])
    s1=pd.concat([s1,s2])
    
#  
shp_contorno_res
cv2.destroyAllWindows()

In [ ]:
np.array(shapes[0].get("coordinates")[0])

In [ ]:
shp_contorno_res

In [ ]:
# Polygon([[map_d(x[0],0,W,minx,maxx),map_d(x[1],H,0,miny,maxy)] for x in box])
# Polygon([[ for x in contours[i]]))

In [ ]:
shp_contorno_res.to_file(r"E:\rastermunicpial/atlacomulcocapa_c.shp")
# s1.to_file(r"E:\rastermunicpial/naucalpan_r.shp")

In [ ]:
# import glob
# filename=glob.glob(r"E:\rastermunicpial/*shp")
# for file in filename:
#     df=gpd.read_file(file)
#     df[df["area"]>1000].to_file(file)
